# Code snippets for accessing and saving various metadata from/to  `.tif` files using `tifffile`

In [2]:
import tifffile

# Leica Matrix Screener `.ome.tif` file acquired on SP5

In [2]:
mscreener = tifffile.TiffFile("C:/Users/Volker/Data/AndreAlves_Subset/chamber--U02--V05/field--X00--Y01/image--L0000--S00--U02--V05--J08--E00--O00--X00--Y01--T0000--Z01--C00.ome.tif")

In [3]:
mscreener.pages.pages[0].is_lsm

False

In [4]:
mscreener.pages.pages[0].is_ome

True

In [5]:
mscreener.ome_metadata

'<?xml version="1.0" encoding="UTF-8" standalone="no"?><!-- Warning: this comment is an OME-XML metadata block, which contains crucial dimensional parameters and other important metadata. Please edit cautiously (if at all), and back up the original data before doing so. For more information, see the OME-TIFF web site: http://loci.wisc.edu/ome/ome-tiff.html. --><OME xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns="http://www.openmicroscopy.org/Schemas/OME/2008-09" xmlns:Bin="http://www.openmicroscopy.org/Schemas/BinaryFile/2008-09" xmlns:CA="http://www.openmicroscopy.org/Schemas/CA/2008-09" xmlns:SA="http://www.openmicroscopy.org/Schemas/SA/2008-09" xmlns:SPW="http://www.openmicroscopy.org/Schemas/SPW/2008-09" xmlns:STD="http://www.openmicroscopy.org/Schemas/STD/2008-09" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2008-09 http://www.openmicroscopy.org/Schemas/OME/2008-09/ome.xsd http://www.openmicroscopy.org/Schem

So the above is a string reprentation of an `.xml` file. If you `pip install xmltodict` you can parse this into nested dictionaries.

In [8]:
import xmltodict
leica_dict = xmltodict.parse(mscreener.ome_metadata)
leica_dict.keys()

odict_keys(['OME'])

In [10]:
leica_dict['OME'].keys()

odict_keys(['@xmlns:xs', '@xmlns', '@xmlns:Bin', '@xmlns:CA', '@xmlns:SA', '@xmlns:SPW', '@xmlns:STD', '@xmlns:xsi', '@xsi:schemaLocation', '@UUID', 'Experiment', 'Experimenter', 'Group', 'Instrument', 'Image', 'SemanticTypeDefinitions', 'CustomAttributes', 'StructuredAnnotations'])

In [14]:
leica_dict['OME'].keys()

odict_keys(['@xmlns:xs', '@xmlns', '@xmlns:Bin', '@xmlns:CA', '@xmlns:SA', '@xmlns:SPW', '@xmlns:STD', '@xmlns:xsi', '@xsi:schemaLocation', '@UUID', 'Experiment', 'Experimenter', 'Group', 'Instrument', 'Image', 'SemanticTypeDefinitions', 'CustomAttributes', 'StructuredAnnotations'])

In [15]:
leica_dict['OME']['Image']['Pixels']

OrderedDict([('@DimensionOrder', 'XYZCT'),
             ('@PixelType', 'uint8'),
             ('@BigEndian', 'false'),
             ('@SizeX', '1024'),
             ('@SizeY', '1024'),
             ('@SizeZ', '1'),
             ('@SizeC', '1'),
             ('@SizeT', '1'),
             ('@PhysicalSizeX', '0.44563245356794E0'),
             ('@PhysicalSizeY', '0.44563245356794E0'),
             ('@PhysicalSizeZ', '0.0'),
             ('@TimeIncrement', '0.0'),
             ('@ID', 'urn:lsid:loci.wisc.edu:Pixels:ows581'),
             ('TiffData',
              OrderedDict([('@FirstC', '0'),
                           ('@FirstZ', '0'),
                           ('UUID',
                            OrderedDict([('@FileName',
                                          'image--L0000--S00--U00--V00--J00--E00--O00--X00--Y00--T0000--Z00--C00.ome.tif'),
                                         ('#text',
                                          'urn:uuid:8ba158a3-fc33-11e8-8ac3-eccd6d63b66a')]

In [25]:
print(f"PhysicalSize X: {leica_dict['OME']['Image']['Pixels']['@SizeX']}")
print(f"PhysicalSize X: {leica_dict['OME']['Image']['Pixels']['@SizeY']}")
print(f"PhysicalSize X: {leica_dict['OME']['Image']['Pixels']['@PhysicalSizeX']}")
print(f"PhysicalSize Y: {leica_dict['OME']['Image']['Pixels']['@PhysicalSizeY']}")
print(f"Stage X: {leica_dict['OME']['Image']['Pixels']['Plane']['StagePosition']['@PositionX']}")
print(f"Stage Y: {leica_dict['OME']['Image']['Pixels']['Plane']['StagePosition']['@PositionY']}")

PhysicalSize X: 1024
PhysicalSize X: 1024
PhysicalSize X: 0.44563245356794E0
PhysicalSize Y: 0.44563245356794E0
Stage X: 0.4456462555080E-1
Stage Y: 0.6673977694440E-1


Sometimes you may not want the `tifffile` and `xmltodict` dependencies.
In that case, you can just read the whole tiff file as one large byte array and search within (note that this approach only works reliably if the key is only present exactly once):

In [26]:
def getKey(bytearr, bytekey):
    indexPos = bytearr.find(bytekey)
    indexStart = bytearr.find(b'"', indexPos)  # find leading quotation mark
    indexStop = bytearr.find(b'"', indexStart + 1)  # find trailing quotation mark
    val = bytearr[indexStart + 1:indexStop]
    return val.decode('utf-8')

with open("C:/Users/Volker/Data/AndreAlves_Subset/chamber--U05--V02/field--X00--Y00/image--L0000--S00--U05--V02--J08--E00--O00--X00--Y00--T0000--Z00--C00.ome.tif", "rb") as f:
    bstrg = f.read()
print(getKey(bstrg, b"PositionX"))
print(getKey(bstrg, b"PositionY"))
print(getKey(bstrg, b"Power"))

0.7168467707040E-1
0.3921968390280E-1
0.5300006103888180E2


# 3i Lattice Lightsheet operated and capturing with Janelia Labview software

In [6]:
lls = tifffile.TiffFile("C:/Users/Volker/Dropbox/Github/Lattice_Lightsheet_Deskew_Deconv/testset/drp1_dendra2_test_1_CamA_ch0_stack0000_488nm_0000000msec_0018218290msecAbs.tif")

c:\users\volker\anaconda3\envs\spimenv\lib\site-packages\tifffile\tifffile.py:4267: UserWarning: tag 32781: coercing invalid ASCII to bytes
  warnings.warn('tag %i: coercing invalid ASCII to bytes' % code)


Note that we already got a warning about tiff tag 32781 being malformed ...

In [7]:
lls.pages.pages

In [8]:
tags = lls.pages.pages[0].tags

###  Print all tags

In [9]:
for t in tags:
    print(f"{t}: {tags[t]}")

ImageWidth: TiffTag 256 ImageWidth  1H @278928  256
ImageLength: TiffTag 257 ImageLength  1H @278928  512
BitsPerSample: TiffTag 258 BitsPerSample  1H @278928  16
Compression: TiffTag 259 Compression  1H @278928  NONE
PhotometricInterpretation: TiffTag 262 PhotometricInterpretation  1H @278928  MINISBLACK
FillOrder: TiffTag 266 FillOrder  1H @278928  MSB2LSB
StripOffsets: TiffTag 273 StripOffsets  1I @278928  (16558,)
Orientation: TiffTag 274 Orientation  1H @278928  TOPLEFT
SamplesPerPixel: TiffTag 277 SamplesPerPixel  1H @278928  1
RowsPerStrip: TiffTag 278 RowsPerStrip  1H @278928  512
StripByteCounts: TiffTag 279 StripByteCounts  1I @278928  (262144,)
XResolution: TiffTag 282 XResolution  2I @278924  (1073741824, 1073741824)
YResolution: TiffTag 283 YResolution  2I @278932  (1073741824, 1073741824)
PlanarConfiguration: TiffTag 284 PlanarConfiguration  1H @278948  CONTIG
ResolutionUnit: TiffTag 296 ResolutionUnit  1H @278948  NONE
Software: TiffTag 305 Software  29s @278940  Nationa

 Turns out that the __str__() of the tag doesn't show us the value of the tag, but a shortened summary of all fields 

In [13]:
tags['ImageID'].__str__()

'TiffTag 32781 ImageID  16550B @8  00 00 00 02 00 00 00 0a 00 00 00 08 41 6e 61 '

Need to look at the `.value` attribute instead:

In [17]:
tags['ImageID'].code

32781

In [18]:
tags['ImageID'].value

b'\x00\x00\x00\x02\x00\x00\x00\n\x00\x00\x00\x08Analysis\x00\x00\x00\tCCDBinROI\x00\x00\x00\x07Channel\x00\x00\x00\rImagePosition\x00\x00\x00\tImgNumber\x00\x00\x00\x0cMaxMinCounts\x00\x00\x00\x0fPixValuePerVolt\x00\x00\x00\x1cScaledPixValuePerAbsPixValue\x00\x00\x00\x08Settings\x00\x00\x00\x07Version\x00\x00\x00\x90\x7f\xff\xff\xff\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@H\x1a\x8f@\x00\x00\x00@F\xbd\xd4@\x00\x00\x00\x00\x00\x008\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x03\x01\x00\x00\x03\x81\x00\x00\x05\x00\x00\x00\x04\x80@\x88\x08\x00\x

In [16]:
lls.pages.pages[0].imagelength

512

In [17]:
lls.pages.pages[0].imagewidth

256

In [18]:
lls.pages.pages[0].imagedepth

1

In [21]:
lls.asarray().shape

(76, 512, 256)

# ImageJ metadata

The information below was posted to Chris Gohlke on this thread
https://forum.image.sc/t/python-copy-all-metadata-from-one-multipage-tif-to-another/26597/8

I'm copying it here for my own reference.


Quote:
Reading selected metadata from an existing file and creating an ImageJ hyperstack compatible file from scratch using that metadata is possible with tifffile but some knowledge of the undocumented ImageJ hyperstack format is required.

For example, using the confocal-series.tif file from the Fiji samples:

In [1]:
from tifffile import TiffFile, imwrite
from tifffile.tifffile import imagej_description_metadata, transpose_axes

with TiffFile('confocal-series.tif') as tif:
    assert tif.is_imagej
    # print detailed information about the file
    print(tif.__str__(detail=2))
    # get image resolution from TIFF tags
    tags = tif.pages[0].tags
    x_resolution = tags['XResolution'].value
    y_resolution = tags['YResolution'].value
    resolution_unit = tags['ResolutionUnit'].value
    # parse ImageJ metadata from the ImageDescription tag
    ij_description = tags['ImageDescription'].value
    ij_description_metadata = imagej_description_metadata(ij_description)
    # get ImageJ app metadata
    ij_metadata = tags['IJMetadata'].value
    # read the whole image stack and get the axes order
    series = tif.series[0]
    ij_hyperstack = series.asarray()
    ij_hyperstack_axes = series.axes

# process ij_hyperstack array; make sure not to change the array type or shape
...

ImportError: cannot import name 'imwrite'

 Beware that tifffile writes little-endian TIFF files by default while ImageJ writes big-endian.
 
      

It is possible to create a TIFF file of individual 2D images, but that will not create an ImageJ hyperstack compatible file:

In [ ]:
with tifffile.TiffWriter('file.tiff') as tif:
    for image in image_stack:
       tif.save(image, contiguous=False, **other_parameters)

# tif file from save individual file during NIS Elements Scan Large Area

In [3]:
tile  = tifffile.TiffFile("/home/hilsenst/Desktop/tile_x001_y007.tif")

In [4]:
tile.epics_metadata

In [5]:
tile.imagej_metadata

In [6]:
tile.metaseries_metadata

In [7]:
tile.series

In [13]:
tags = tile.pages.pages[0].tags

In [14]:
[print(t) for t in tags]

TiffTag 256 ImageWidth @10020600 SHORT @10020608 2044
TiffTag 257 ImageLength @10020612 SHORT @10020620 2060
TiffTag 258 BitsPerSample @10020624 SHORT @10020632 16
TiffTag 259 Compression @10020636 SHORT @10020644 LZW
TiffTag 262 PhotometricInterpretation @10020648 SHORT @10020656 MINISBLACK
TiffTag 266 FillOrder @10020660 SHORT @10020668 MSB2LSB
TiffTag 273 StripOffsets @10020672 LONG[1030] @10024988 (8, 9497, 19028, 28548,
TiffTag 274 Orientation @10020684 SHORT @10020692 TOPLEFT
TiffTag 277 SamplesPerPixel @10020696 SHORT @10020704 1
TiffTag 278 RowsPerStrip @10020708 SHORT @10020716 2
TiffTag 279 StripByteCounts @10020720 LONG[1030] @10020868 (9489, 9531, 9520, 9
TiffTag 281 MaxSampleValue @10020732 SHORT @10020740 65535
TiffTag 284 PlanarConfiguration @10020744 SHORT @10020752 CONTIG
TiffTag 65325 @10020756 DOUBLE @10029132 2623684.261686001
TiffTag 65326 @10020768 DOUBLE @10029116 0.644888355182203
TiffTag 65327 @10020780 DOUBLE @10029124 1.0
TiffTag 65328 @10020792 DOUBLE @10029

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [23]:
tile.ome_metadata

## Tif file from NIS Elements. Results of Job script exported with `export nd2 to tif`


In [16]:
nd2totiff  = tifffile.TiffFile("/home/hilsenst/Desktop/seq0000xy01c1.tif")

In [20]:
tags =nd2totiff.pages.pages[0].tags

In [21]:
[print(t) for t in tags]

TiffTag 256 ImageWidth @8494326 SHORT @8494334 2044
TiffTag 257 ImageLength @8494338 SHORT @8494346 2060
TiffTag 258 BitsPerSample @8494350 SHORT @8494358 16
TiffTag 259 Compression @8494362 SHORT @8494370 NONE
TiffTag 262 PhotometricInterpretation @8494374 SHORT @8494382 MINISBLACK
TiffTag 266 FillOrder @8494386 SHORT @8494394 MSB2LSB
TiffTag 270 ImageDescription @8494398 ASCII[5064] @8567372 <?xml version="1.0" 
TiffTag 273 StripOffsets @8494410 LONG[1030] @8498726 (8, 8184, 16360, 24536, 3
TiffTag 274 Orientation @8494422 SHORT @8494430 TOPLEFT
TiffTag 277 SamplesPerPixel @8494434 SHORT @8494442 1
TiffTag 278 RowsPerStrip @8494446 SHORT @8494454 2
TiffTag 279 StripByteCounts @8494458 LONG[1030] @8494606 (8176, 8176, 8176, 817
TiffTag 281 MaxSampleValue @8494470 SHORT @8494478 65535
TiffTag 284 PlanarConfiguration @8494482 SHORT @8494490 CONTIG
TiffTag 65325 @8494494 DOUBLE @8502846 25400.32065966446
TiffTag 65326 @8494506 DOUBLE @8502854 0.644888355182203
TiffTag 65327 @8494518 DOUB

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [22]:
nd2totiff.ome_metadata

'<?xml version="1.0" encoding="UTF-8"?>\r\n<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2015-01" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2015-01 http://www.openmicroscopy.org/Schemas/OME/2015-01/ome.xsd">\r\n   <Experimenter ID="Experimenter:0" Institution="" LastName=""/>\r\n   <Instrument ID="Instrument:0">\r\n      <!--this instrument tag is add because of backward compatibility with NIS elements 4.30-->\r\n      <LightSource ID="LightSource:0">\r\n         <Laser/>\r\n      </LightSource>\r\n      <Objective CalibratedMagnification="1.000000" ID="Objective:0" LensNA="0.450000" NominalMagnification="1"/>\r\n   </Instrument>\r\n   <Instrument ID="Instrument:1">\r\n      <Microscope Model="Ti Microscope"/>\r\n      <Detector ID="Detector:0"/>\r\n      <Objective Correction="PlanApo" ID="Objective:1" LensNA="0.450000" Model="Plan Apo ? 10x" NominalMagnification="10" WorkingDistance="4000.000000"/>\r\n 

In [26]:
import xmltodict
nd2tiff_dict = xmltodict.parse(nd2totiff.ome_metadata)
nd2tiff_dict.keys()

odict_keys(['OME'])

In [31]:
nd2tiff_dict['OME']['Image']['Pixels']

OrderedDict([('@DimensionOrder', 'XYCZT'),
             ('@ID', 'Pixels:0'),
             ('@PhysicalSizeX', '0.644888'),
             ('@PhysicalSizeY', '0.644888'),
             ('@SizeC', '1'),
             ('@SizeT', '1'),
             ('@SizeX', '2044'),
             ('@SizeY', '2060'),
             ('@SizeZ', '1'),
             ('@Type', 'uint16'),
             ('Channel',
              OrderedDict([('@AcquisitionMode', 'WideField'),
                           ('@Color', '16777215'),
                           ('@EmissionWavelength', '525'),
                           ('@ID', 'Channel:0'),
                           ('@Name', 'Mono'),
                           ('@PinholeSize', '-1.000000'),
                           ('DetectorSettings',
                            OrderedDict([('@Binning', '1x1'),
                                         ('@ID', 'Detector:0')]))])),
             ('TiffData', None),
             ('Plane',
              OrderedDict([('@DeltaT', '25400.320313'),
 